In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import sqlite3 as sq
import geodict
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
def percent(x, y):
    return (x/y)*100

Occupation:

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  

*load the following query: JEQOutlook_Occupation*

+ go to "Draft Mode"
+ Select the "Occupation" dataset
+ Select the following dimensions: Region, Occupation  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Census Places"   
+ Under "Occupation" expand "Total-All Occupations" and select the built groups in Jobs EQ as described below, leaving Total as well  
 
Occupations to Select:  
(SOC Codes)  
+ Total - All Occupations  
(Custom Created Groups) 
+ SnapshotOcc 11: Healthcare Practitioners and Technical (29: Healthcare Practitioners and Technical Occupations)  
+ SnapshotOcc 11: Healthcare Support (31: Healthcare Support Occupations)  
+ SnapshotOcc 11: Protective Service (33: Protective Service Occupations)  
+ SnapshotOcc 11: Production (51: Production Occupations)  
+ SnapshotOcc 11: Transportation and Material Moving (53: Transportation and Material Moving Occupations)   
+ SnapshotOcc 11: Business & Financial (11: Management & 13: Business & Financial Operations)  
+ SnapshotOcc 11: Computer, Mathematical, & Engineering (15: Computer & Mathematical, 17: Architecture & Engineering, & 19: Life, Physical, & Social Science)  
+ SnapshotOcc 11: Education, Legal, Social, Arts, & Media (21: Community & Social Service, 23: Legal, 25: Educational Instruction & Library, & 27: Art, Design, Entertainment, Sports, & Media)  
+ SnapshotOcc 11: Food, Property, & Personal Services (35: Food Preparation & Serving, 37: Building & Grounds Cleaning & Maintenance, & 39: Personal Care & Service)  
+ SnapshotOcc 11: Sales & Office Support (41: Sales & Related & 43: Office & Administrative Support)  
+ SnapshotOcc 11: Natural Resources, Construction, & Maintenance (45: Farming Fishing & Forestry, 47: Construction & Extraction, & 49: Installation Maintenance & Repair)

Then put in draft mode & select the following values: "Employment (Place of Work)", "Employment (Place of Residence)", "Annual Wage (Mean)", "Annual Wage (Median)", "Location Quotient", Demand - Total(Forecast) (3 Years), Demand - Growth Rate (Forecast)(3 Years)". Put region on the left side and occupation at the top.

*Resume here after loading query.*

Once it is completed, there are a few things that are just easier to do in excel. Open and delete the top row that says "Occupation Data", and delete the FIPS column. Rename the Region column to NAME, add NAME above this to create a double header. Delete the data notes at the bottom. Unmerge all of the occupation cells at the top and simply copy/paste the occupation category, ensuring that each column has a group header above the value header. Finally, select all numerical data and format as "general".  

Save in "Data Downloads" as: "JobsEQOccupationalEmploymentforOutlook_AllGeos.csv"

In [2]:
#import data and set a variable keeping track of the most recent download's year and quarter
data = pd.read_csv('../../Data Downloads/JobsEQ_OccupationalEmploymentforOutlook_AllGeos.csv')
recentquarter = 'Q3 2022'
data.head(2)

,NAME,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,SnapshotOcc 11: Production,SnapshotOcc 11: Production.1,SnapshotOcc 11: Production.2,SnapshotOcc 11: Production.3,SnapshotOcc 11: Production.4,SnapshotOcc 11: Production.5,SnapshotOcc 11: Production.6,SnapshotOcc 11: Healthcare Support,SnapshotOcc 11: Healthcare Support.1,SnapshotOcc 11: Healthcare Support.2,SnapshotOcc 11: Healthcare Support.3,SnapshotOcc 11: Healthcare Support.4,SnapshotOcc 11: Healthcare Support.5,SnapshotOcc 11: Healthcare Support.6,SnapshotOcc 11: Protective Services,SnapshotOcc 11: Protective Services.1,SnapshotOcc 11: Protective Services.2,SnapshotOcc 11: Protective Services.3,SnapshotOcc 11: Protective Services.4,SnapshotOcc 11: Protective Services.5,SnapshotOcc 11: Protective Services.6,SnapshotOcc 11: Business & Financial,SnapshotOcc 11: Business & Financial.1,SnapshotOcc 11: Business & Financial.2,SnapshotOcc 11: Business & Financial.3,SnapshotOcc 11: Business & Financial.4,SnapshotOcc 11: Business & Financial.5,SnapshotOcc 11: Business & Financial.6,SnapshotOcc 11: Sales & Office Support,SnapshotOcc 11: Sales & Office Support.1,SnapshotOcc 11: Sales & Office Support.2,SnapshotOcc 11: Sales & Office Support.3,SnapshotOcc 11: Sales & Office Support.4,SnapshotOcc 11: Sales & Office Support.5,SnapshotOcc 11: Sales & Office Support.6,SnapshotOcc 11: Transportation and Material Moving,SnapshotOcc 11: Transportation and Material Moving.1,SnapshotOcc 11: Transportation and Material Moving.2,SnapshotOcc 11: Transportation and Material Moving.3,SnapshotOcc 11: Transportation and Material Moving.4,SnapshotOcc 11: Transportation and Material Moving.5,SnapshotOcc 11: Transportation and Material Moving.6,"SnapshotOcc 11: Food, Property, & Personal Services","SnapshotOcc 11: Food, Property, & Personal Services.1","SnapshotOcc 11: Food, Property, & Personal Services.2","SnapshotOcc 11: Food, Property, & Personal Services.3","SnapshotOcc 11: Food, Property, & Personal Services.4","SnapshotOcc 11: Food, Property, & Personal Services.5","SnapshotOcc 11: Food, Property, & Personal Services.6","SnapshotOcc 11: Computer, Mathematical, & Engineering","SnapshotOcc 11: Computer, Mathematical, & Engineering.1","SnapshotOcc 11: Computer, Mathematical, & Engineering.2","SnapshotOcc 11: Computer, Mathematical, & Engineering.3","SnapshotOcc 11: Computer, Mathematical, & Engineering.4","SnapshotOcc 11: Computer, Mathematical, & Engineering.5","SnapshotOcc 11: Computer, Mathematical, & Engineering.6",SnapshotOcc 11: Healthcare Practitioners and Technical,SnapshotOcc 11: Healthcare Practitioners and Technical.1,SnapshotOcc 11: Healthcare Practitioners and Technical.2,SnapshotOcc 11: Healthcare Practitioners and Technical.3,SnapshotOcc 11: Healthcare Practitioners and Technical.4,SnapshotOcc 11: Healthcare Practitioners and Technical.5,SnapshotOcc 11: Healthcare Practitioners and Technical.6,"SnapshotOcc 11: Education, Legal, Social, Arts, & Media","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.1","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.2","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.3","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.4","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.5","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.6","SnapshotOcc 11: Natural Resources, Construction, & Maintenance","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.1","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.2","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.3","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.4","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.5","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.6"
0,NAME,Empl (Place of Work),Empl (Place of Residence),Annual Mean Wage1,Ann 50th %ile Wage1

In [3]:
#set the index as the NAME column, transpose, rename the double NAME to value (recall the double in the download excel formatting), then reset index without dropping
data = data.set_index('NAME').transpose().rename(columns = {'NAME': 'Value'}).reset_index(drop = False)
data.head(2)

NAME,index,Value,USA,Tennessee,MPO,GNRC,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Big Rock CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Gladeville CDP, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Greenvale CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Norene CDP, TN","Oak Grove CDP, TN (Sumner )","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Statesville CDP, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Tuckers Crossroads CDP, TN","Vanleer town, TN","Walnut Grove CDP, TN (Sumner )","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee"
0,Total - All Occupations,Empl (Place of Work),162472851.6,3454675.749,1092474.644,1161776.732,1204731.632,46.77625746,5095.347207,1354.046814,17812.64096,56.23123535,9.245202135,16.47046353,57172.88463,258.4331226,41.84481809,12.64808424,1114.942565,580.1041512,55675.52324,23441.5902,1899.579277,19.42627555,293.3180848,792.9323171,13828.42154,1226.137126,465.2695067,1022.601123,5.450833144,2011.323994,1080.499582,85853.63863,24695.20645,355.7348892,14432.73481,1.314209026,1286.220478,298.949005,0.574247913,2360.232916,25657.65174,1010.118148,22248.40506,36375.99018,472.7717858,1507.488391,15.87074102,19799.18377,2354.628092,77535.58938,537565.3941,57.56496371,1137.159845,2915.890658,6.722630275,65.56634464,1252.356346,468.89792,299.8537576,1629.024537,10006.14577,382.6430825,468.4196654,149.8494601,516.9566196,17.98303688,40088.59287,11614.45259,10895.08142,5.249829193,192.4034522,1979.560118,94.40247289,54.92235884,35.73391588,189.4505715,640.3178639,2122.676226,961.1414927,924.30509,5182.252584,11125.07124,571506.2215,20489.61045,1818.80663,6463.954228,42954.89964,66598.40899,25998.95028,152790.2894,3400.874166,67705.30158,2360.26154,164459.555,67059.42699
1,Total - All Occupations.1,Empl (Place of Residence),157234754.4,3294067.451,1008766.42,1113253.29,1165041.248,399.6137489,2911.15788,1596.001818,1998.352262,188.8160078,132.9221617,58.95618499,25888.37274,878.8978438,343.2729784,137.1782438,610.2288477,2198.028206,62918.9964,21948.81388,2074.73173,207.365539,1058.664361,131.719171,7843.784645,763.3741075,340.3137747,511.4122823,75.0737748,5602.973641,2630.145552,50248.31241,20297.78443,745.2407447,9584.55114,99.36979149,3247.33009,3593.213955,36.46582827,5366.307287,31887.86685,1484.561705,19494.2092,18977.35154,704.9423315,3177.79185,95.46699838,20443.37311,2325.62326,77048.50652,4

In [4]:
#strip the index and value columns; remove blank space that would be heading or trailing the strings
data['index'] = data['index'].str.strip()
data['Value'] = data['Value'].str.strip()

In [5]:
#set these strings as the index, so that you can transpose the df and fill the null values with zeros to convert the datatype for calculations
data = data.set_index(['index', 'Value']).transpose().fillna(0)

In [6]:
#make a list of these numerical columns, and convert them to float datatype
cols = data.columns
data[cols] = data[cols].astype(float)

In [8]:
data.head()

index,Total - All Occupations,Total - All Occupations.1,Total - All Occupations.2,Total - All Occupations.3,Total - All Occupations.4,Total - All Occupations.5,Total - All Occupations.6,SnapshotOcc 11: Production,SnapshotOcc 11: Production.1,SnapshotOcc 11: Production.2,SnapshotOcc 11: Production.3,SnapshotOcc 11: Production.4,SnapshotOcc 11: Production.5,SnapshotOcc 11: Production.6,SnapshotOcc 11: Healthcare Support,SnapshotOcc 11: Healthcare Support.1,SnapshotOcc 11: Healthcare Support.2,SnapshotOcc 11: Healthcare Support.3,SnapshotOcc 11: Healthcare Support.4,SnapshotOcc 11: Healthcare Support.5,SnapshotOcc 11: Healthcare Support.6,SnapshotOcc 11: Protective Services,SnapshotOcc 11: Protective Services.1,SnapshotOcc 11: Protective Services.2,SnapshotOcc 11: Protective Services.3,SnapshotOcc 11: Protective Services.4,SnapshotOcc 11: Protective Services.5,SnapshotOcc 11: Protective Services.6,SnapshotOcc 11: Business & Financial,SnapshotOcc 11: Business & Financial.1,SnapshotOcc 11: Business & Financial.2,SnapshotOcc 11: Business & Financial.3,SnapshotOcc 11: Business & Financial.4,SnapshotOcc 11: Business & Financial.5,SnapshotOcc 11: Business & Financial.6,SnapshotOcc 11: Sales & Office Support,SnapshotOcc 11: Sales & Office Support.1,SnapshotOcc 11: Sales & Office Support.2,SnapshotOcc 11: Sales & Office Support.3,SnapshotOcc 11: Sales & Office Support.4,SnapshotOcc 11: Sales & Office Support.5,SnapshotOcc 11: Sales & Office Support.6,SnapshotOcc 11: Transportation and Material Moving,SnapshotOcc 11: Transportation and Material Moving.1,SnapshotOcc 11: Transportation and Material Moving.2,SnapshotOcc 11: Transportation and Material Moving.3,SnapshotOcc 11: Transportation and Material Moving.4,SnapshotOcc 11: Transportation and Material Moving.5,SnapshotOcc 11: Transportation and Material Moving.6,"SnapshotOcc 11: Food, Property, & Personal Services","SnapshotOcc 11: Food, Property, & Personal Services.1","SnapshotOcc 11: Food, Property, & Personal Services.2","SnapshotOcc 11: Food, Property, & Personal Services.3","SnapshotOcc 11: Food, Property, & Personal Services.4","SnapshotOcc 11: Food, Property, & Personal Services.5","SnapshotOcc 11: Food, Property, & Personal Services.6","SnapshotOcc 11: Computer, Mathematical, & Engineering","SnapshotOcc 11: Computer, Mathematical, & Engineering.1","SnapshotOcc 11: Computer, Mathematical, & Engineering.2","SnapshotOcc 11: Computer, Mathematical, & Engineering.3","SnapshotOcc 11: Computer, Mathematical, & Engineering.4","SnapshotOcc 11: Computer, Mathematical, & Engineering.5","SnapshotOcc 11: Computer, Mathematical, & Engineering.6",SnapshotOcc 11: Healthcare Practitioners and Technical,SnapshotOcc 11: Healthcare Practitioners and Technical.1,SnapshotOcc 11: Healthcare Practitioners and Technical.2,SnapshotOcc 11: Healthcare Practitioners and Technical.3,SnapshotOcc 11: Healthcare Practitioners and Technical.4,SnapshotOcc 11: Healthcare Practitioners and Technical.5,SnapshotOcc 11: Healthcare Practitioners and Technical.6,"SnapshotOcc 11: Education, Legal, Social, Arts, & Media","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.1","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.2","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.3","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.4","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.5","SnapshotOcc 11: Education, Legal, Social, Arts, & Media.6","SnapshotOcc 11: Natural Resources, Construction, & Maintenance","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.1","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.2","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.3","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.4","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.5","SnapshotOcc 11: Natural Resources, Construction, & Maintenance.6"
Value,Empl (Place of Work),Empl (Place of Residence),Annual Mean Wage1,Ann 50th %ile Wage1,

In [9]:
#list occupations and calculate the percent of the total for all (except for total all occupations)
occs = ['SnapshotOcc 11: Healthcare Practitioners and Technical', 'SnapshotOcc 11: Healthcare Support', 
        'SnapshotOcc 11: Protective Services',
        'SnapshotOcc 11: Production', 'SnapshotOcc 11: Transportation and Material Moving', 'SnapshotOcc 11: Business & Financial', 
        'SnapshotOcc 11: Computer, Mathematical, & Engineering', 'SnapshotOcc 11: Education, Legal, Social, Arts, & Media', 
        'SnapshotOcc 11: Food, Property, & Personal Services', 
        'SnapshotOcc 11: Sales & Office Support', 'SnapshotOcc 11: Natural Resources, Construction, & Maintenance']
for occ in occs:
    data[('{}'.format(occ), 'Percent of Total')] = percent(data[('{}'.format(occ), 'Empl (Place of Work)')], 
                                                           data[('Total - All Occupations', 'Empl (Place of Work)')])

In [10]:
#transpose this df now that you've made calculations, and reset the index without dropping
data = data.transpose().reset_index(drop = False)

In [11]:
#create a dataframe from the "index" column (occupation) where you split the string on the "." (.1, .2, because of duplicates), expand on delimiter
boop = data['index'].str.split(pat = ".", expand = True)
#set new column data['Occupation'] as the first item in this other dataframe (position 0)
data['Occupation'] = boop[0]

In [12]:
#drop the original index column
data.drop(columns = 'index', inplace = True)

In [13]:
#list the columns of the dataframe, as these will be made into the value variables in a melt function
cols = data.columns
#melt the data (wide to long)
data = data.melt(var_name = 'NAME', id_vars = ['Value', 'Occupation'], value_vars = cols, value_name = 'placeholder')
data.head(2)

,Value,Occupation,NAME,placeholder
0,Empl (Place of Work),Total - All Occupations,USA,162472851.6
1,Empl (Place of Residence),Total - All Occupations,USA,157234754.4


In [14]:
#it will be easiest to get the occupation name up top to rename these, so pivot the table so it's ready to go
data = data.pivot(index = ['Occupation', 'NAME'], columns = 'Value')['placeholder']
#reset the index without dropping, set the index as "Occupation", and transpose to rename
data = data.reset_index(drop = False).set_index('Occupation').transpose()

In [15]:
#rename occupations
data = data.rename(columns = {'SnapshotOcc 11: Healthcare Practitioners and Technical': 'Healthcare Practitioners and Technical', 
                              'SnapshotOcc 11: Healthcare Support': 'Healthcare Support', 
                              'SnapshotOcc 11: Protective Services': 'Protective Services',
                              'SnapshotOcc 11: Production': 'Production', 
                              'SnapshotOcc 11: Transportation and Material Moving': 'Transportation and Material Moving', 
                              'SnapshotOcc 11: Business & Financial': 'Business & Financial', 
                              'SnapshotOcc 11: Computer, Mathematical, & Engineering': 'Computer, Mathematical, & Engineering', 
                              'SnapshotOcc 11: Education, Legal, Social, Arts, & Media': 'Education, Legal, Social, Arts, & Media', 
                              'SnapshotOcc 11: Food, Property, & Personal Services': 'Food, Property, & Personal Services', 
                              'SnapshotOcc 11: Sales & Office Support': 'Sales & Office Support', 
                              'SnapshotOcc 11: Natural Resources, Construction, & Maintenance': 'Natural Resources, Construction, & Maintenance'}
                  ).transpose().reset_index(drop = False)

In [16]:
occs = list(data['Occupation'].unique())

In [17]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos before resetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus).transpose().reset_index()

In [18]:
#rename the columns
data = data.rename(columns = {'Total 3 Year Forecast Demand': '3 Year Forecast Demand', 
                              'Empl (Place of Work)': 'Employment', 'LQ': 'Location Quotient', 'Annual Mean Wage1': 'Average Annual Wages', 
                              'Ann 50th %ile Wage1': 'Median Annual Wages'})

In [19]:
#multiply the forecast growth rate *100 to fit the other formatting and drop the demand as we won't use right now
data['3 Year Forecast Growth Rate'] = data['3 Year Forecast Growth Rate'] * 100
#data = data.drop(columns = ['3 Year Forecast Demand', 'Empl (Place of Residence)', 'Average Annual Wages'])

In [20]:
#this will set a year and quarter column to the variable set upon data import
data['Year and Quarter'] = recentquarter

In [21]:
cols = ['3 Year Forecast Growth Rate', 'Median Annual Wages', 'Employment', 'Location Quotient', 'Percent of Total']
data[cols] = data[cols].astype(float)

In [22]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [23]:
#final check
data.head()

Value,NAME,Occupation,3 Year Forecast Growth Rate,Median Annual Wages,Average Annual Wages,Empl (Place of Residence),Employment,Location Quotient,Percent of Total,3 Year Forecast Demand,Year and Quarter,GEO_ID,Source
0,"Adams city, Tennessee",Business & Financial,2.757140,90200.0,100800.0,66.926935,5.014536,0.807563,10.720260,1.531045,Q3 2022,1600000US4700200,JobsEQ
1,"Ashland City town, Tennessee",Business & Financial,3.081871,82600.0,91900.0,198.729745,532.581345,0.787378,10.452307,159.615766,Q3 2022,1600000US4702180,JobsEQ
2,"Belle Meade city, Tennessee",Business & Financial,4.955700,99600.0,114100.0,366.659444,143.423408,0.797917,10.592205,48.75898,Q3 2022,1600000US4704620,JobsEQ
3,"Berry Hill city, Tennessee",Business & Financial,2.103759,92200.0,105000.0,511.7498,2506.680401,1.060088,14.072480,711.25895,Q3 2022,1600000US4705140,JobsEQ
4,"Brentwood city, Tennessee",Business & Financial,8.506231,93800.0,106600.0,6952.550854,12454.822940,1.641038,21.784493,4649.909708,Q3 2022,1600000US4708280,JobsEQ


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NAME                         888 non-null    object 
 1   Occupation                   888 non-null    object 
 2   3 Year Forecast Growth Rate  888 non-null    float64
 3   Median Annual Wages          888 non-null    float64
 4   Average Annual Wages         888 non-null    object 
 5   Empl (Place of Residence)    888 non-null    object 
 6   Employment                   888 non-null    float64
 7   Location Quotient            888 non-null    float64
 8   Percent of Total             814 non-null    float64
 9   3 Year Forecast Demand       888 non-null    object 
 10  Year and Quarter             888 non-null    object 
 11  GEO_ID                       816 non-null    object 
 12  Source                       888 non-null    object 
dtypes: float64(5), objec

In [25]:
#export to the SQLite database as only the JobsEQ quarterly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_OccupationalEmployment_3YROutlook', conn, if_exists = 'replace', index = False)

888

Industry:  

To retrieve data:
+ Log into JobsEQ and navigate to the "Data Explorer" function  

*load the following query: JEQOutlook_Industry*

+ Select the "Industry" dataset, quarterly  
+ Select the following dimensions: Region, Industry  
+ Under "Region" select "USA", "Tennessee", GNRC (13), MPO (7), GNRC Region (14), and then under GNRC Region sub all "Counties" and "Census Places"   
+ Under "Industry" expand "Total-All Industries" and select the built groups in Jobs EQ as described below, leaving Total as well  


+ Total - All Industries  
(Custom Created Groups *begin to type them as they appear here and they will appear in the search bar, if not then remake the custom group as described below*)  
+ Snapshot 11: Construction (23: Construction)  
+ Snapshot 11: Manufacturing (31: Manufacturing)  
+ Snapshot 11: Information (51: Information)  
+ Snapshot 11: Public Administration (92: Public Administration)  
+ Snapshot 11: Other/Unclassified (81: Other, Except Public Administration, 99: Unclassified)  
+ Snapshot 11: Leisure & Hospitality (71: Arts, Entertainment, & Recreation, 72: Accommodation & Food Services)   
+ Snapshot 11: Education & Health Services (61: Education, 62: Healthcare & Social Assistance)   
+ Snapshot 11: Professional & Business Services (54: Professional, Scientific, & Technical, 55: Management of Companies & Enterprises, 56: Administrative & Support & Waste Management & Remediation)   
+ Snapshot 11: Financial Activities (52: Finance & Insurance, 53: Real Estate & Rental & Leasing)   
+ Snapshot 11: Trade, Transportation & Utilities (42: Wholesale Trade, 44-45: Retail Trade, 48-49: Transportation & Warehousing, 22: Utilities)  
+ Snapshot 11: Natural Resources & Mining (11: Agricultural Activities, Forestry, Fishing, & Hunting, 21: Mining, Quarrying, Oil & Gas) 
+ Snapshot: White Collar  
+ Snapshot: Blue Collar  
+ Snapshot 4: Government  
+ Snapshot 4: Retail & Hospitality  
+ Snapshot 4: Professional Services  
+ Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics  

Go into draft mode and select the following values: "Employment (4Q Avg)", "Avg Ann Wages per Worker", "Location Quotient (4Q Avg)", Forecast Growth Demand (3 Years), Forecast Growth Rate (3 Years)". Have region on the left hand side and industry up top.   

*Resume here after loading query and downloading data.*

Once it is completed, there are a few things that are just easier to do in excel. Open and delete the top row that says "Industry Data", and delete the FIPS column. Rename the Region column to NAME, add NAME above this to create a double header. Delete the data notes at the bottom. Unmerge all of the industry cells at the top and simply copy/paste the industry category, ensuring that each column has a group header above the value header. Finally, select all numerical data and format as "general". Save as "JobsEQIndustryEmployment_AllGeos.csv"

In [100]:
#import data and set a variable keeping track of the most recent download's year and quarter
data = pd.read_csv('../../Data Downloads/JobsEQ_IndustryEmploymentforOutlook_AllGeos.csv')
recentquarter = 'Q3 2022'
data.head(2)

,NAME,Total - All Industries,Total - All Industries.1,Total - All Industries.2,Total - All Industries.3,Total - All Industries.4,Snapshot 11: Construction,Snapshot 11: Construction.1,Snapshot 11: Construction.2,Snapshot 11: Construction.3,Snapshot 11: Construction.4,Snapshot 11: Manufacturing,Snapshot 11: Manufacturing.1,Snapshot 11: Manufacturing.2,Snapshot 11: Manufacturing.3,Snapshot 11: Manufacturing.4,Snapshot 11: Information,Snapshot 11: Information.1,Snapshot 11: Information.2,Snapshot 11: Information.3,Snapshot 11: Information.4,Snapshot 11: Public Administration,Snapshot 11: Public Administration.1,Snapshot 11: Public Administration.2,Snapshot 11: Public Administration.3,Snapshot 11: Public Administration.4,Snapshot 11: Other/Unclassified,Snapshot 11: Other/Unclassified.1,Snapshot 11: Other/Unclassified.2,Snapshot 11: Other/Unclassified.3,Snapshot 11: Other/Unclassified.4,Snapshot 11: Leisure & Hospitality,Snapshot 11: Leisure & Hospitality.1,Snapshot 11: Leisure & Hospitality.2,Snapshot 11: Leisure & Hospitality.3,Snapshot 11: Leisure & Hospitality.4,Snapshot 11: Education & Health Services,Snapshot 11: Education & Health Services.1,Snapshot 11: Education & Health Services.2,Snapshot 11: Education & Health Services.3,Snapshot 11: Education & Health Services.4,Snapshot 11: Professional & Business Services,Snapshot 11: Professional & Business Services.1,Snapshot 11: Professional & Business Services.2,Snapshot 11: Professional & Business Services.3,Snapshot 11: Professional & Business Services.4,Snapshot 11: Financial Activities,Snapshot 11: Financial Activities.1,Snapshot 11: Financial Activities.2,Snapshot 11: Financial Activities.3,Snapshot 11: Financial Activities.4,"Snapshot 11: Trade, Transportation, & Utilities","Snapshot 11: Trade, Transportation, & Utilities.1","Snapshot 11: Trade, Transportation, & Utilities.2","Snapshot 11: Trade, Transportation, & Utilities.3","Snapshot 11: Trade, Transportation, & Utilities.4",Snapshot 11: Natural Resources & Mining,Snapshot 11: Natural Resources & Mining.1,Snapshot 11: Natural Resources & Mining.2,Snapshot 11: Natural Resources & Mining.3,Snapshot 11: Natural Resources & Mining.4,Snapshot: White Collar,Snapshot: White Collar.1,Snapshot: White Collar.2,Snapshot: White Collar.3,Snapshot: White Collar.4,Snapshot: Blue Collar,Snapshot: Blue Collar.1,Snapshot: Blue Collar.2,Snapshot: Blue Collar.3,Snapshot: Blue Collar.4,Snapshot 4: Government,Snapshot 4: Government.1,Snapshot 4: Government.2,Snapshot 4: Government.3,Snapshot 4: Government.4,Snapshot 4: Retail & Hospitality,Snapshot 4: Retail & Hospitality.1,Snapshot 4: Retail & Hospitality.2,Snapshot 4: Retail & Hospitality.3,Snapshot 4: Retail & Hospitality.4,Snapshot 4: Professional Services,Snapshot 4: Professional Services.1,Snapshot 4: Professional Services.2,Snapshot 4: Professional Services.3,Snapshot 4: Professional Services.4,"Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics","Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics.1","Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics.2","Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics.3","Snapshot 4: Industrial, Manufacturing, Utilities, & Logistics.4"
0,NAME,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast Growth Demand,3 Year Forecast Growth Rate,Empl (4Q Mov Avg),Avg Ann Wages per Worker,LQ (4Q Mov Avg),3 Year Forecast G

In [101]:
#set the index as the NAME column, transpose, rename the double NAME to value (recall the double in the download excel formatting), then reset index without dropping
data = data.set_index('NAME').transpose().rename(columns = {'NAME': 'Value'}).reset_index(drop = False)
data.head(2)

NAME,index,Value,USA,Tennessee,MPO,GNRC,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Big Rock CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Gladeville CDP, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Greenvale CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Norene CDP, TN","Oak Grove CDP, TN (Sumner )","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Statesville CDP, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Tuckers Crossroads CDP, TN","Vanleer town, TN","Walnut Grove CDP, TN (Sumner )","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee"
0,Total - All Industries,Empl (4Q Mov Avg),162472883.7,3454683.751,1092474.644,1161776.732,1204731.631,46.77625746,5095.347207,1354.046814,17812.64096,56.23123535,9.245202136,16.47046353,57172.88463,258.4331226,41.84481809,12.64808424,1114.942565,580.1041512,55675.52324,23441.5902,1899.579277,19.42627555,293.3180848,792.9323171,13828.42154,1226.137126,465.2695067,1022.601123,5.450833143,2011.323994,1080.499582,85853.63863,24695.20645,355.7348892,14432.73481,1.314209024,1286.220478,298.949005,0.574247912,2360.232916,25657.65174,1010.118148,22248.40506,36375.99018,472.7717858,1507.488391,15.87074102,19799.18377,2354.628092,77535.58938,537565.3941,57.5649637,1137.159845,2915.890658,6.722630276,65.56634464,1252.356346,468.89792,299.8537576,1629.024537,10006.14577,382.6430825,468.4196654,149.8494601,516.9566196,17.98303688,40088.59287,11614.45259,10895.08142,5.249829193,192.4034522,1979.560118,94.40247289,54.92235885,35.73391588,189.4505715,640.3178639,2122.676226,961.1414927,924.30509,5182.252584,11125.07124,571506.2215,20489.61046,1818.806639,6463.954245,42954.89964,66598.40905,25998.95023,152790.2894,3400.874169,67705.30143,2360.261542,164459.5549,67059.42696
1,Total - All Industries.1,Avg Ann Wages per Worker,68501.25988,60975.24347,69788.14701,68203.43816,67737.22351,48343.28866,53793.94926,73730.44022,71800.97586,49844.76568,91163.57136,49632.20504,82751.7974,47010.00486,59536.09147,42173.4533,47248.97629,53499.41588,46839.38788,55913.88127,47027.53411,52941.8257,45996.30949,55588.52634,47549.85617,49453.17979,49635.07339,37568.14995,54511.90196,75391.48539,73802.42003,81819.61857,54113.70582,47669.46011,73016.4824,44618.548,48532.09508,53904.52616,38901.7746,46976.85337,54175.08694,51786.84473,57395.21478,53486.64421,52584.27631,52645.40513,54658.11251,53027.36911,55129.15736,56635.74015

In [102]:
#strip the index and value columns; remove blank space that would be heading or trailing the strings
data['index'] = data['index'].str.strip()
data['Value'] = data['Value'].str.strip()

In [103]:
#set these strings as the index, so that you can transpose the df and fill the null values with zeros to convert the datatype for calculations
data = data.set_index(['index', 'Value']).transpose().fillna(0)

In [104]:
#make a list of these numerical columns, and convert them to float datatype
cols = data.columns
data[cols] = data[cols].astype(float)

In [105]:
#list industries and calculate the percent of the total for all (except for total all industries)
inds = ['Snapshot 11: Construction', 'Snapshot 11: Manufacturing', 'Snapshot 11: Information', 'Snapshot 11: Public Administration', 
        'Snapshot 11: Other/Unclassified', 'Snapshot 11: Leisure & Hospitality', 'Snapshot 11: Education & Health Services', 
        'Snapshot 11: Professional & Business Services', 'Snapshot 11: Financial Activities', 'Snapshot 11: Trade, Transportation, & Utilities', 
        'Snapshot 11: Natural Resources & Mining']
for ind in inds:
    data[('{}'.format(ind), 'Percent of Total')] = percent(data[('{}'.format(ind), 'Empl (4Q Mov Avg)')], data[('Total - All Industries', 'Empl (4Q Mov Avg)')])

In [106]:
#transpose this df now that you've made calculations, and reset the index without dropping
data = data.transpose().reset_index(drop = False)
data.head()

NAME,index,Value,USA,Tennessee,MPO,GNRC,GNRC Region,"Adams city, TN","Ashland City town, TN","Belle Meade city, TN","Berry Hill city, TN","Bethpage CDP, TN","Big Rock CDP, TN","Bransford CDP, TN","Brentwood city, TN","Burns town, TN","Castalian Springs CDP, TN","Cedar Hill city, TN","Charlotte town, TN","Christiana CDP, TN","Clarksville city, TN","Columbia city, TN","Coopertown town, TN","Cottontown CDP, TN","Cross Plains city, TN","Cumberland City town, TN","Dickson city, TN","Dover city, TN","Eagleville city, TN","Erin city, TN","Fairfield CDP, TN","Fairview city, TN","Forest Hills city, TN","Franklin city, TN","Gallatin city, TN","Gladeville CDP, TN","Goodlettsville city, TN","Graball CDP, TN","Greenbrier town, TN","Green Hill CDP, TN","Greenvale CDP, TN","Hartsville/Trousdale County, TN","Hendersonville city, TN","Kingston Springs town, TN","La Vergne city, TN","Lebanon city, TN","McEwen city, TN","Millersville city, TN","Mitchellville city, TN","Mount Juliet city, TN","Mount Pleasant city, TN","Murfreesboro city, TN","Nashville-Davidson metropolitan government (balance), TN","New Deal CDP, TN","New Johnsonville city, TN","Nolensville town, TN","Norene CDP, TN","Oak Grove CDP, TN (Sumner )","Oak Hill city, TN","Orlinda city, TN","Pegram town, TN","Pleasant View city, TN","Portland city, TN","Ridgetop city, TN","Rockvale CDP, TN","Rural Hill CDP, TN","Shackle Island CDP, TN","Slayden town, TN","Smyrna town, TN","Springfield city, TN","Spring Hill city, TN","Statesville CDP, TN","Tennessee Ridge town, TN","Thompson's Station town, TN","Tuckers Crossroads CDP, TN","Vanleer town, TN","Walnut Grove CDP, TN (Sumner )","Walterhill CDP, TN","Watertown city, TN","Waverly city, TN","Westmoreland town, TN","White Bluff town, TN","White House city, TN","Cheatham County, Tennessee","Davidson County, Tennessee","Dickson County, Tennessee","Houston County, Tennessee","Humphreys County, Tennessee","Maury County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Rutherford County, Tennessee","Stewart County, Tennessee","Sumner County, Tennessee","Trousdale County, Tennessee","Williamson County, Tennessee","Wilson County, Tennessee"
0,Total - All Industries,Empl (4Q Mov Avg),1.624729e+08,3.454684e+06,1.092475e+06,1.161777e+06,1.204732e+06,46.776257,5095.347207,1354.046814,17812.640960,56.231235,9.245202,16.470464,57172.884630,258.433123,41.844818,12.648084,1114.942565,580.104151,55675.523240,23441.590200,1899.579277,19.426276,293.318085,792.932317,13828.421540,1226.137126,465.269507,1022.601123,5.450833,2011.323994,1080.499582,85853.63863,24695.206450,355.734889,14432.734810,1.314209,1286.220478,298.949005,0.574248,2360.232916,25657.651740,1010.118148,22248.405060,36375.990180,472.771786,1507.488391,15.870741,19799.183770,2354.628092,77535.589380,537565.394100,57.564964,1137.159845,2915.890658,6.722630,65.566345,1252.356346,468.897920,299.853758,1629.024537,10006.145770,382.643082,468.419665,149.849460,516.956620,17.983037,40088.592870,11614.452590,10895.081420,5.249829,192.403452,1979.560118,94.402473,54.922359,35.733916,189.450571,640.317864,2122.676226,961.141493,924.305090,5182.252584,11125.071240,571506.221500,20489.610460,1818.806639,6463.954245,42954.899640,66598.409050,25998.950230,152790.289400,3400.874169,67705.301430,2360.261542,164459.554900,67059.426960
1,Total - All Industries.1,Avg Ann Wages per Worker,6.850126e+04,6.097524e+04,6.978815e+04,6.820344e+04,6.773722e+04,48343.288660,53793.949260,73730.440220,71800.975860,49844.765680,91163.571360,49632.205040,82751.797400,47010.004860,59536.091470,42173.453300,47248.976290,53499.415880,46839.387880,55913.881270,47027.534110,52941.825700,45996.309490,55588.526340,47549.856170,49453.179790,49635.073390,37568.149950,54511.901960,75391.485390,73802.420030,81819.61857,54113.705820,47669.460110,73016.482400,44618.548000,48532.095080,53904.526160,38901.774600,46976.853370,54175.086940,51786.844730,57395.214780,53486.644210,52584.276310,52645.405130,54658.112510,53

In [107]:
#create a dataframe from the "index" column (industry) where you split the string on the "." (.1, .2, because of duplicates), expand on delimiter
boop = data['index'].str.split(pat = ".", expand = True)
#set new column data['Industry'] as the first item in this other dataframe (position 0)
data['Industry'] = boop[0]

In [109]:
#drop the original index column
data.drop(columns = 'index', inplace = True)

In [110]:
#list the columns of the dataframe, as these will be made into the value variables in a melt function
cols = data.columns
#melt the data (wide to long)
data = data.melt(var_name = 'NAME', id_vars = ['Value', 'Industry'], value_vars = cols, value_name = 'placeholder')
data.head(2)

,Value,Industry,NAME,placeholder
0,Empl (4Q Mov Avg),Total - All Industries,USA,1.624729e+08
1,Avg Ann Wages per Worker,Total - All Industries,USA,6.850126e+04


In [111]:
#it will be easiest to get the industry name up top to rename these, so pivot the table so it's ready to go
data = data.pivot(index = ['Industry', 'NAME'], columns = 'Value')['placeholder']
#reset the index without dropping, set the index as "Industry", and transpose to rename
data = data.reset_index(drop = False).set_index('Industry').transpose()

In [112]:
#rename columns
data = data.rename(columns = {'Total - All Industries': 'Total', 'Snapshot 11: Construction': 'Construction',
                              'Snapshot 11: Manufacturing': 'Manufacturing', 'Snapshot 11: Information': 'Information', 
                              'Snapshot 11: Public Administration': 'Public Administration', 
                              'Snapshot 11: Other/Unclassified': 'Other/Unclassified', 
                              'Snapshot 11: Leisure & Hospitality': 'Leisure & Hospitality', 
                              'Snapshot 11: Education & Health Services': 'Education & Health Services', 
                              'Snapshot 11: Professional & Business Services': 'Professional & Business Services', 
                              'Snapshot 11: Financial Activities': 'Financial Activities', 
                              'Snapshot 11: Trade, Transportation, & Utilities': 'Trade, Transportation, & Utilities', 
                              'Snapshot 11: Natural Resources & Mining': 'Natural Resources & Mining'
                             }).transpose().reset_index(drop = False)

In [114]:
#remove the CDPs from the geographies, then transpose and use dictionary from custom module to rename geos before resetting the index
data = data.loc[~data['NAME'].str.contains('CDP')]
data = data.set_index('NAME').transpose()
data = data.rename(columns = tofullcensus).transpose().reset_index()

In [115]:
#rename the columns
data = data.rename(columns = {'3 Year Forecast Growth Demand': '3 Year Forecast Demand', 'Avg Ann Wages per Worker': 'Average Annual Wages', 
                              'Empl (4Q Mov Avg)': 'Employment', 'LQ (4Q Mov Avg)': 'Location Quotient'})

In [116]:
#multiply the forecast growth rate *100 to fit the other formatting and drop the demand as we won't use right now
data['3 Year Forecast Growth Rate'] = data['3 Year Forecast Growth Rate'] * 100
data = data.drop(columns = '3 Year Forecast Demand')

In [117]:
#this will set a year and quarter column to the variable set upon data import
data['Year and Quarter'] = recentquarter

In [118]:
#remove geography names that contain CDP, make sure that the index has no name for feather format export, and drop the index to have standard index format for feather
data = data.rename_axis(None, axis=1).reset_index(drop = True)

In [120]:
cols = ['3 Year Forecast Growth Rate', 'Average Annual Wages', 'Employment', 'Location Quotient', 'Percent of Total']
data[cols] = data[cols].astype(float)

In [121]:
#map to geoid dictionary from module and add source
data['GEO_ID'] = data['NAME'].map(geotogeoid)
data['Source'] = 'JobsEQ'

In [122]:
#final check
data.head()

,NAME,Industry,3 Year Forecast Growth Rate,Average Annual Wages,Employment,Location Quotient,Percent of Total,Year and Quarter,GEO_ID,Source
0,"Adams city, Tennessee",Construction,2.969643,64055.96486,1.655657,0.596043,3.539525,Q3 2022,1600000US4700200,JobsEQ
1,"Ashland City town, Tennessee",Construction,2.578872,58282.22036,135.852261,0.448979,2.666202,Q3 2022,1600000US4702180,JobsEQ
2,"Belle Meade city, Tennessee",Construction,2.172766,77723.58825,31.024150,0.385832,2.291217,Q3 2022,1600000US4704620,JobsEQ
3,"Berry Hill city, Tennessee",Construction,2.187311,78029.51461,813.531405,0.769092,4.567158,Q3 2022,1600000US4705140,JobsEQ
4,"Brentwood city, Tennessee",Construction,6.849030,86795.43310,2449.524934,0.721480,4.284417,Q3 2022,1600000US4708280,JobsEQ


In [123]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1332 entries, 0 to 1331
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   NAME                         1332 non-null   object 
 1   Industry                     1332 non-null   object 
 2   3 Year Forecast Growth Rate  1332 non-null   float64
 3   Average Annual Wages         1332 non-null   float64
 4   Employment                   1332 non-null   float64
 5   Location Quotient            1332 non-null   float64
 6   Percent of Total             814 non-null    float64
 7   Year and Quarter             1332 non-null   object 
 8   GEO_ID                       1224 non-null   object 
 9   Source                       1332 non-null   object 
dtypes: float64(5), object(5)
memory usage: 104.2+ KB


In [124]:
#export to the SQLite database as only the JobsEQ quarterly data
conn = sq.connect('../../Outputs/Labor_Economy.db')
data.to_sql('JobsEQ_IndustryEmployment_3YROutlook', conn, if_exists = 'replace', index = False)

1332